In [4]:
import duckdb
import pandas as pd
import wget

import pandas as pd
import unicodedata
import re
import duckdb
import wget
from google.cloud import storage
import os

def remove_accents(text):
    """
    Remove accent marks from input text while preserving the base characters.
    Also handles special characters like Đ/đ.
    """
    if not isinstance(text, str):
        return text
        
    special_chars = {
        'Đ': 'D', 'đ': 'd', 'Ł': 'L', 'ł': 'l', 'Ø': 'O', 'ø': 'o',
        'Ŧ': 'T', 'ŧ': 't', 'Æ': 'AE', 'æ': 'ae', 'Œ': 'OE', 'œ': 'oe',
        'ß': 'ss'
    }
    
    for char, replacement in special_chars.items():
        text = text.replace(char, replacement)
    
    normalized_text = unicodedata.normalize('NFKD', text)
    result = ''.join(c for c in normalized_text if not unicodedata.category(c).startswith('Mn'))
    
    return result


# Bring in nba player lookup table to map the cleaned names to player IDs. Same player IDs from the NBA API.
filename = 'nba_player_lookup.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
# Read in the nba_player_lookup csv
nba_player_lookup_df = pd.read_csv(filename)

# Clean each player's full name
nba_player_lookup_df["player_name"] = nba_player_lookup_df["player_name"].apply(remove_accents)

In [10]:
query = """
SELECT player_name, COUNT(*) AS COUNT FROM nba_player_lookup_df GROUP BY player_name
HAVING COUNT(*) > 1
ORDER BY COUNT(*) DESC
"""

duckdb.query(query).df()

,player_name,COUNT
0,George Johnson,3
1,Charles Smith,3
2,Dee Brown,2
3,Michael Smith,2
4,Gerald Henderson,2
5,Charles Jones,2
6,George King,2
7,Tony Mitchell,2
8,Walker Russell,2
9,Chris Smith,2


In [2]:
import pandas as pd
from attach_player_ids import attach_player_ids

# Test with sample data
test_df = pd.DataFrame({
    'Player': ['LeBron James', 'Stephen Curry'],
    'Points': [25, 28]
})

result = attach_player_ids(test_df, 'Player')
result

,player_id,player_name,Points
0,201939,Stephen Curry,28
1,2544,LeBron James,25
